# Homegate.ch scraping

In [12]:
# importing necessary packages
# using bs and requests modules for scraping

from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd


In [13]:
# We define page number here for looping pagination
cur_page = 1


# After carefully isnpecting website, realized apt rental advertisement goes in 2 categories:
# Premium one (paid subscription) and simple ones (without subscription) therefore create 2 empty list

premium = []
simple = []

# Creating function for changing page numbers and While True loop has to run until it breaks
def getLink(page):
    return f"https://www.homegate.ch/rent/apartment/city-geneva/matching-list?ep={page}"
    
while True:
    print("Page ->", cur_page)
    link = getLink(cur_page)
    
    res = requests.get(link)
    
    
## we parse HTML with Beautiful Soup
    bs = BeautifulSoup(res.text, features='html.parser')
    
## we define 2 variable a for premium and b for simple annoucements
## and with find_all() we returns all div containers with mentioned class names that match our filters

    a = bs.find_all('div', {'class': 'ListItemTopPremium_item_K9dLF'})
    b = bs.find_all('div', {'class': 'ListItem_item_1GcIZ'})

## if we get zero results then we break the while loop defined earlier (we have 21 pages of results)

    if len(a) == 0 and len(b) == 0:
        break
        
## otherwise we run for loop for findings in a and b then append according to the empty list created earlier

    for offer in a:
        premium.append(offer)
    for offer in b:
        simple.append(offer)

    
# printing results and incrementing page number by one 

    print(len(premium), len(simple))
    cur_page += 1

# Defining function block with result dic keys price,size,rooms,address and for each of them we use try except to get 
# within span tag with mentioned class name info needed  and add them to created list

def extractPremiumInfo(block):
    result = {
        'price': None,
        'size': None,
        'rooms': None,
        'address': None
    }
    try:
        price = block.find('span', {'class': 'ListItemPrice_price_1o0i3'}).find_all('span')[1].text
        result['price'] = price
    except:
        pass

    try:
        m2 = block.find('span', {'class': 'ListItemLivingSpace_value_2zFir'}).text
        result['size'] = m2
    except:
        pass

    try:
        rn = block.find('span', {'class': 'ListItemRoomNumber_value_Hpn8O'}).text
        result['rooms'] = rn
    except:
        pass

    address = block.find('div', {'class': 'ListItemTopPremium_data_3i7Ca'})
    if address is None:
        address = block.find('div', {'class': 'ListItem_data_18_z_'})

    address = address.find_all('p')[1].text

    result['address'] = address

    return result

# Again with for loop we go over simple and premium list and append results to fnish list.

finish = []

for i in premium:
    finish.append(extractPremiumInfo(i))

for i in simple:
    finish.append(extractPremiumInfo(i))

print(f"Found {len(finish)}apartments")

Page -> 1
20 0
Page -> 2
21 19
Page -> 3
21 39
Page -> 4
21 59
Page -> 5
21 79
Page -> 6
21 99
Page -> 7
21 119
Page -> 8
21 139
Page -> 9
21 159
Page -> 10
21 179
Page -> 11
21 199
Page -> 12
21 219
Page -> 13
21 239
Page -> 14
21 259
Page -> 15
21 279
Page -> 16
21 299
Page -> 17
21 319
Page -> 18
21 339
Page -> 19
21 359
Page -> 20
21 379
Page -> 21
21 380
Page -> 22
Found 401apartments


In [6]:
df = pd.DataFrame(finish)
df.to_csv('Geneva_listings_src.csv', index=False, encoding='utf-8')